In [1]:
import itertools
import json
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import os
import random
import Simulation
import time
import networkx as nx

from Algorithms.LAIM import *
from Algorithms.TIM  import *
from Algorithms.Proposed import *
from Algorithms.Heuristics import random_solution
from Diffusion.Model import *
from scipy.stats import arcsine
from tqdm import tqdm

sns.set_style("darkgrid")
RESULTS_PATH = os.path.join("..", "out", "results")
META_PATH = os.path.join(RESULTS_PATH, "meta.json")

uniform   = lambda: random.random(); uniform.__name__ = "uniform"
polarized = lambda: arcsine.rvs();   polarized.__name__ = "polarized"

def get_meta_count():
    with open(META_PATH) as f:
        data = json.load(f)
    return data["count"]

def increment_meta_count():
    count = get_meta_count()
    with open(META_PATH, "w") as f:
        json.dump('{"count": %d}' % count + 1, f)

In [2]:
# Dependent variables.
topologies = ["amazon"]#, "dblp", "youtube", "wiki"]
algorithms = [random_solution]
seed_sizes = list(range(0, 1000+1, 100)); seed_sizes[0] = 1
opinion_distrs = [uniform, polarized]
use_communities = [False, True]

# Constant parameters.
n_trials = 2#100
t_horizon = 100
random_seed = 16785407


## Experiment Design
Here we are interested in comparing the performance/efficacy of four baseline algorithms (i.e., `IRIE`, `TIM+`, `LAIM`, and `FastLAIM`) against our proposed solution (i.e., `OpinionDegree`) for opinion maximization under the BIC model.

More specifically, we are interested in this question under a few scenarios:
* Community-agnostic vs. Community-aware opinion initialization.
* Uniform opinion distribution vs. Polarized opinion distribution.

In [5]:
# Initialize the results Pandas.DataFrame object for collecting all data.
results = pd.DataFrame()
parameter_combinations = itertools.product(topologies, algorithms, opinion_distrs, use_communities)

for (topo_code, alg, opinion_distr, use_comm) in parameter_combinations:
    new_results = Simulation.run(
        topo_code          = topo_code, 
        algorithm          = alg, 
        seed_sizes         = seed_sizes, 
        time_horizon       = t_horizon, 
        n_trials           = n_trials, 
        opinion_distr_func = opinion_distr,
        random_seed        = random_seed,
        use_communities    = use_comm,
        pbar_desc          = "topo={}_alg={}_comm={}_{}".format(topo_code, alg, use_comm, opinion_distr.__name__)
    )
    results = results.append(new_results)

topo=amazon_alg=<function random_solution at 0x7fc74dd76378>_comm=False_uniform:   0%|          | 0/22 [00:00<?, ?it/s]

FileNotFoundError: [Errno 2] No such file or directory: 'topos/com-amazon.ungraph.txt.gz'

In [4]:
results.to_csv("dummy.csv")